# Artificial Neural Network

### Importing the libraries

In [73]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [74]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [75]:
tf.__version__

'2.7.0'

## Part 1 - Data Preprocessing

### Importing the dataset

In [118]:
dataset = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Part2_ArtificialNeuralNetworks/Churn_Modelling.csv')
X = dataset.iloc[:, 3:-1].values
y = dataset.iloc[:, -1].values

### Encoding categorical data

Label Encoding the "Gender" column

In [119]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:,2] = le.fit_transform(X[:,2])

One Hot Encoding the "Geography" column

In [120]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder 
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

### Splitting the dataset into the Training set and Test set

In [121]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1)

### Feature Scaling

In [122]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
print(X_train)

[[-0.99850112  1.71490137 -0.57273139 ... -1.55337352  0.97725852
   0.42739449]
 [ 1.00150113 -0.58312392 -0.57273139 ... -1.55337352 -1.02327069
  -1.02548708]
 [-0.99850112  1.71490137 -0.57273139 ...  0.64376017  0.97725852
  -0.94479772]
 ...
 [ 1.00150113 -0.58312392 -0.57273139 ...  0.64376017  0.97725852
  -0.14096853]
 [ 1.00150113 -0.58312392 -0.57273139 ...  0.64376017  0.97725852
   0.01781218]
 [-0.99850112  1.71490137 -0.57273139 ...  0.64376017 -1.02327069
  -1.15822478]]


## Part 2 - Building the ANN

### Initializing the ANN

In [123]:
ann = tf.keras.models.Sequential()

### Adding the input layer and the first hidden layer

In [124]:
ann.add(tf.keras.layers.Dense(units = 6, activation = 'relu'))

### Adding the second hidden layer

In [125]:
ann.add(tf.keras.layers.Dense(units = 6, activation = 'relu'))

### Adding the output layer

In [126]:
ann.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid'))

## Part 3 - Training the ANN

### Compiling the ANN

In [127]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the ANN on the Training set

In [128]:
ann.fit(X_train, y_train, batch_size = 32, epochs = 25)

Epoch 1/25
250/250 [==============================] - 1s 2ms/step - loss: 0.6967 - accuracy: 0.5659
Epoch 2/25
250/250 [==============================] - 0s 2ms/step - loss: 0.5134 - accuracy: 0.8006
Epoch 3/25
250/250 [==============================] - 0s 2ms/step - loss: 0.4557 - accuracy: 0.8080
Epoch 4/25
250/250 [==============================] - 0s 2ms/step - loss: 0.4328 - accuracy: 0.8119
Epoch 5/25
250/250 [==============================] - 0s 2ms/step - loss: 0.4212 - accuracy: 0.8183
Epoch 6/25
250/250 [==============================] - 0s 2ms/step - loss: 0.4111 - accuracy: 0.8270
Epoch 7/25
250/250 [==============================] - 0s 2ms/step - loss: 0.3991 - accuracy: 0.8351
Epoch 8/25
250/250 [==============================] - 0s 2ms/step - loss: 0.3865 - accuracy: 0.8432
Epoch 9/25
250/250 [==============================] - 0s 2ms/step - loss: 0.3766 - accuracy: 0.8494
Epoch 10/25
250/250 [==============================] - 0s 2ms/step - loss: 0.3689 - accuracy: 0.8528

## Part 4 - Making the predictions and evaluating the model

### Predicting the result of a single observation

**Homework**

Use our ANN model to predict if the customer with the following informations will leave the bank: 

Geography: France

Credit Score: 600

Gender: Male

Age: 40 years old

Tenure: 3 years

Balance: \$ 60000

Number of Products: 2

Does this customer have a credit card? Yes

Is this customer an Active Member: Yes

Estimated Salary: \$ 50000

So, should we say goodbye to that customer?

In [129]:
obs = sc.transform([[1, 0, 0, 600, 1, 40, 3, 62000, 2, 1, 1, 50000]])
ann.predict(obs)

array([[0.04082641]], dtype=float32)

**Solution**

Therefore, our ANN model predicts that this customer stays in the bank!

**Important note 1:** Notice that the values of the features were all input in a double pair of square brackets. That's because the "predict" method always expects a 2D array as the format of its inputs. And putting our values into a double pair of square brackets makes the input exactly a 2D array.

**Important note 2:** Notice also that the "France" country was not input as a string in the last column but as "1, 0, 0" in the first three columns. That's because of course the predict method expects the one-hot-encoded values of the state, and as we see in the first row of the matrix of features X, "France" was encoded as "1, 0, 0". And be careful to include these values in the first three columns, because the dummy variables are always created in the first columns.

### Predicting the Test set results

In [132]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred), 1), y_test.reshape(len(y_test), 1)), 1)[1:10])

[[0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]]


### Making the Confusion Matrix

In [133]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
acc = accuracy_score(y_test, y_pred)
print(cm)
print(acc)

[[1546   39]
 [ 241  174]]
0.86
